In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
data=pd.read_csv('Cluster.csv')
data_main=data.copy()

## Dimentionality Reduction using PCA

In [ ]:
model_PCA = PCA(n_components=3)
PCs = model_PCA.fit_transform(data)

Assign a DataFrame to PCs to change list into dataframe for easy use

In [ ]:
df=pd.DataFrame(PCs,columns=['PC1','PC2','PC3'])
df.head()

## Getting the coeffcinet of each column of data corresponding to PC1,PC2,PC3

In [ ]:
model_PCA.components_

In [ ]:
pd.DataFrame(model_PCA.components_)

In [ ]:
pd.DataFrame(model_PCA.components_,columns=data.columns, index=['PC1','PC2','PC3']).transpose()

## Logic Behind PCs--- Mean=0

In [ ]:
round(df.describe(),2)

# Distortion Calculation

In [ ]:
distortion=[]
 
for i in range(1,15):
    model = KMeans(n_clusters=i,init='k-means++',n_init=10,max_iter=50,random_state=1234)
    model.fit(df)
    distortion.append(model.inertia_)
    print("Count of Clusters:{} ".format(i) , "Distortion:{}".format(round(model.inertia_,2)))
    
#plot the distortion   
px.line(x=range(1,15),y=distortion,title="Distortion Vs Count of Clusters")
#make cluster and calculate the cluster distortion 
#1 cluster distortion is 90 but it get reduced as we make many clusters and finally it becomes flat as we can see in the graph

# Creating Final Cluster

In [ ]:
model = KMeans(n_clusters=7,init='k-means++',n_init=10,max_iter=50,random_state=1234)
cluster_number=model.fit_predict(df)

In [ ]:
len(cluster_number)

In [ ]:
cluster_number


In [ ]:
np.unique(cluster_number)

# Assign Cluster Number to Main Data

In [ ]:
data1=data.copy()
data1['cluster_number']=cluster_number.tolist()

In [ ]:
data1['PC1']=df['PC1']
data1['PC2']=df['PC2']
data1['PC3']=df['PC3']

In [ ]:
data1.head()

In [ ]:
fig=px.scatter_3d(data1,x='PC1',y='PC2',z='PC3', color='cluster_number')
fig.show()

## Pivot Table for Feature Importance

In [ ]:
pivot_table=pd.pivot_table(data1,index=['cluster_number'],
                           values=['PC1','PC2','PC3'],
                           aggfunc=[np.min,np.max]
                          
                          
                          )


flat=pd.DataFrame(pivot_table.to_records())
flat

flat.columns = [hdr.replace("('a", "").replace("', '","_").replace("')","") for hdr in flat.columns]
flat


In [ ]:
PC_importance = pd.DataFrame(model_PCA.components_,columns=data.columns,index=['PC1','PC2','PC3'])

In [ ]:
PC_importance.style.background_gradient(cmap='coolwarm')

In [ ]:
# Adding global minimum and maximum

flat['PC1_G_min']=data1['PC1'].min()
flat['PC1_G_max']=data1['PC1'].max()

flat['PC2_G_min']=data1['PC2'].min()
flat['PC2_G_max']=data1['PC2'].max()

flat['PC3_G_min']=data1['PC3'].min()
flat['PC3_G_max']=data1['PC3'].max()


In [ ]:
round(flat.transpose(),2)

## Feature Impact::
1 PC1

  a.Positive --Mat_x,NO,HS,SR_x ---cluster 4
  
  b.negative--Mat_y,wkts,Econ,SR_y,X4w,X5w---cluster 6
  
2 PC2

  a.Positive --Mat_y,Wkts,Econ,SR_y,X4w,Mat_x,NO--cluster 3
  
  b.negative--X100,X50,X4s,X6s--cluster 2& 5
  
3 PC3

  a.Positive --X100,X50,X4s,X6s,X5w--cluster 4
  
  b.negative--Mat_x,NO,HS,Avg_x,SR_x---cluster 1,3

# Write the cluster number to originl dataset

In [ ]:
real_data=pd.read_csv('IPL.csv')

real_data['cluster_number']=cluster_number

real_data.to_csv('IPL_WithCluster.csv',index=False)